### Group Controller

- Load in a roster of Spheros
- Connect to each one and calibrate their position
- Perform some orchestration


In [1]:
# General Python Helper Functions
def readJsonFile(filename):
    '''
        Read a JSON file into a python dict 
    '''
    with open(filename) as data_file:    
        data = json.load(data_file)
    return data

In [2]:
# NEW SPHEROS
import sphero
import time

# Controller Functions
import os
import json
from pprint import pprint

In [5]:
# Use sphero app to get sphero names and MAC addresses
PROJ_ROOT = os.getcwd()
ROSTER = os.path.join(PROJ_ROOT, "roster.json")
roster = readJsonFile(ROSTER)
pprint(roster)

{u'68:86:E7:05:0C:4B': u'Sphero-YYP',
 u'68:86:E7:05:12:26': u'Sphero-RPB',
 u'68:86:E7:05:19:AD': u'Sphero-RYR',
 u'68:86:E7:09:4E:74': u'Sphero-YPR',
 u'68:86:E7:09:A2:FE': u'Sphero-RWR',
 u'68:86:E7:09:A6:FE': u'Sphero-ORG',
 u'68:86:E7:09:A9:28': u'Sphero-GRY'}


In [7]:
# Class for managing multiple spheros
manager = sphero.SpheroManager()

In [8]:
# Initialize Sphero manager with addresses of local spheros
manager._name_cache = roster

# Number of spheros to connect to
NBOTS = len(roster.keys())
print(NBOTS)

In [10]:
# This searches the system cache for all of the robots. 
# It doesn't need the spheros to be turned on yet.
def on_new_sphero(device, NBOTS=NBOTS):
    """
        NBOTS = number of robots in the flock
        Note that this presently refers to a global "manager" object. This method should be moved inside the native
        library later.
    """
    print "Found " + device.bt_name
    
    # Terminate search when all expected bots are found
    if len(manager._spheros.keys()) == NBOTS:
        print "Found all {} spheros".format(NBOTS)
        manager.stop_auto_search()
                 
# Callback
manager.set_sphero_found_cb(on_new_sphero)

# Construct list of devices in system bluetooth collection
# Your bluetooth network cannot contain anything with name of "Sphero-" prefix
devices = []
manager.start_auto_search()

Starts auto search
Found Sphero-GRY
Found Sphero-YPR
Found Sphero-RWR
Found Sphero-ORG
Found Sphero-RYR
Found Sphero-RPB
Found Sphero-YYP
Found all 7 spheros


In [12]:
devices = []
# Verify that these are the robots we are looking for
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)
    devices.append(device) 

Sphero-RPB: 68:86:E7:05:12:26
Sphero-YPR: 68:86:E7:09:4E:74
Sphero-GRY: 68:86:E7:09:A9:28
Sphero-YYP: 68:86:E7:05:0C:4B
Sphero-RWR: 68:86:E7:09:A2:FE
Sphero-ORG: 68:86:E7:09:A6:FE
Sphero-RYR: 68:86:E7:05:19:AD


In [19]:
activeBotNames = [
    "Sphero-RYR"
]

These are experiments on 4/15


In [20]:
# Test notes 4/15
# bots = [ devices[0], devices[4], devices[5]]
bots = [ manager._spheros[name] for name in activeBotNames ]

In [21]:
# Connect team to computer
connect_team(bots)

Sphero-RYR try 0
Connected!


In [24]:
# Make sure everyone's battery is above 1.5
print_team_status(bots)

Sphero-RYR Battery OK | 8.05


In [27]:
for bot in bots:
    bot.set_rgb(50,50,0)

In [38]:
# Manage sphero colors consistently
colors = [
    [255, 0 , 0], # R
    [0, 255 , 0], # G
    [0, 0 , 255], # B
    [255, 0, 255] # Purple
]

# Control Brightness
for color in colors:
    for i, val in enumerate(color):
        color[i] /= 2
        
# Initialization for any team
set_team_back_led(bots, True)
set_team_colors(bots, colors)
print_team_status(bots)

Sphero-RYR Battery OK | 8.04
Received unknown async msg! Header:  (255, 254, 5, 1)


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 415, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 299, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1



In [14]:
# Math Functions
def normalize_angle(angle):
    if angle < 0:
        return 360 + angle
    elif angle > 359:
        return angle - 360
    else:
        return angle

In [37]:
# Team control methods
def connect_team(bots):
    for i, bot in enumerate(bots):  
        bot.disconnect()
        bot.connect()
        
def disconnect_team(bots):
    for i, bot in enumerate(bots):  
        bot.disconnect()
        
def set_team_back_led(bots, status):
    # Bright if true, dim if false
    status = 0xaa if status else 0x00
    
    for bot in bots:  
        bot.set_back_led_output(status)
        
def set_team_colors(bots, colors):
    for i, bot in enumerate(bots):  
        colorTriple = colors[i]
        bot.set_rgb(colorTriple[0], colorTriple[1], colorTriple[2])
        
def print_team_status(bots):
    for bot in bots:
        response = bot.get_power_state()
        print "{} {} | {}".format(bot.bt_name, response.power_state, response.bat_voltage)
        
def roll_sphero(bot, speed, heading, offset):
    """
        Roll robot in in proper direction at a given speed
    """
    bot.roll(speed, normalize_angle(heading + offset))
    
def roll_sphero_team_synchronized(bots, speed, heading, offsets,motionTimeout=2000):
    """
        Move all robots in same direction at shared speed
    """
#     for bot in bots:
#         bot.set_motion_timeout(motionTimeout)
    assert(len(bots) == len(offsets))
    tStart = time.time()
    
    for i, bot in enumerate(bots):
        roll_sphero(bot, speed, heading, offsets[i])
        
    tEnd = time.time()
    
    print("Dispatch Time {}".format(tEnd - tStart) )
    time.sleep(motionTimeout / 1000)  # wait for bots to finish rolling

In [240]:
# angles = [0, 120, 240]
# Demo Reel: show basic 

botAngleOffsets = [
    15, 120, 90, 230         
]

corners = 3
angles = [ i * (360 / corners) for i in range(corners)]


for angle in angles:
    roll_sphero_team_synchronized(bots, 80, angle, botAngleOffsets)
    
    
# roll_sphero_team_synchronized(bots, 100, 30, botAngleOffsets)

Dispatch Time 0.357000112534
Dispatch Time 0.218999862671
Dispatch Time 0.266999959946


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 415, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 299, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1



In [181]:
# code in offsets manually for r,g,b

# direction = 50

direction = 40

speed = 70



    
    
time.sleep(3)

direction2 = direction + 180
    
for i, bot in enumerate(bots):
    roll_sphero(bot, speed, direction2, botAngleOffsets[i])
    

In [177]:
# bot2 = [b for b in manager._spheros.iteritems()][1][1]

In [9]:
bot = [b for b in manager._spheros.iteritems()][1]


In [10]:
bot = bot[1]


In [ ]:
# bot2.connect()



In [11]:
bot.set_rgb(0, 255,0 )



In [2]:
import cv2



In [3]:
cam = cv2.VideoCapture(0)



In [4]:
print cam



<VideoCapture 0000000004A5EE90>


In [5]:
s, im = cam.read() # captures image
cv2.imshow("Test Picture", im) # displays captured image

